In [1]:
import os

os.chdir('/home/jinho93/tmdc/WTe2/mo-alloy/nonpolar/conf1/ps/md/more')

In [4]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

In [22]:
import numpy as np
from maml.utils import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces)
_, df = convert_docs(train_pool)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'force'] = 100

from maml.base import SKLModel
from maml.describers import BispectrumCoefficients
from sklearn.linear_model import LinearRegression
from maml.apps.pes import SNAPotential

element_profile = {'Mo': {'r': 5.0, 'w': 1}, 'W': {'r': 5.0, 'w': 1}, 'Te': {'r': 5.0, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=0.5, twojmax=8, element_profile=element_profile, 
                                   quadratic=False, pot_fit=True)
model = SKLModel(describer=describer, model=LinearRegression())
snap = SNAPotential(model=model)
snap.train(train_structures, train_energies, train_forces, sample_weight=weights)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial
/home/jinho93/miniconda3/envs/mlph/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [23]:
snap.write_param()

['pair_style        snap',
 'pair_coeff        * * SNAPotential.snapcoeff SNAPotential.snapparam Te Mo W']

In [27]:
from maml.apps.pes import EnergyForceStress

snap_loaded = SNAPotential.from_config(param_file='SNAPotential.snapparam', coeff_file='SNAPotential.snapcoeff')
efs_calculator = EnergyForceStress(ff_settings=snap_loaded)
energy, forces, stresses = efs_calculator.calculate([train_structures[1]])[0]

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial


In [28]:
train_forces[1] - forces

array([[ 0.27979442,  0.01782325,  0.63310421],
       [-0.24547336, -0.57390969, -1.10617118],
       [ 0.01216847,  0.26463131, -0.69286571],
       [ 0.35308789,  0.09436893,  0.72469003],
       [-0.73629025,  0.18691734, -0.90410654],
       [ 0.18788008, -1.0962712 , -0.69290261],
       [-0.55676081,  0.21108372, -0.1628396 ],
       [ 0.71920509,  0.49161086,  1.28471646],
       [-0.10367151, -0.22121414, -0.49840986],
       [-0.27214034,  0.49226066,  0.99331104],
       [ 0.66251445, -0.39862958, -0.22066259],
       [ 0.63377511,  0.51491357, -0.88132315],
       [-0.19337644, -0.01007738, -0.64268836],
       [ 0.60221452, -0.32839713,  0.23763696],
       [ 0.17868317, -0.2789173 ,  0.30566515],
       [-0.57397138,  0.41590673, -0.18415211],
       [ 0.21286794, -0.22866563, -0.07212231],
       [-0.13504735, -0.04159042,  0.20795117],
       [-0.02847632,  0.29032393,  0.00384842],
       [-0.20162254, -0.67193236, -0.88184167],
       [ 0.03672308, -0.31635729,  0.056

In [21]:
train_energies[0] - energy


-8.057020828175041